## 0. Import Required Libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

## 1. Load the CSV Data

In [3]:
# Load the CSV data into a DataFrame
df = pd.read_csv('energy_data.csv', delimiter=';')

# Convert the DATEFROM and TIMEFROM columns into a datetime format
df['DATETIME'] = pd.to_datetime(df['DATEFROM'] + ' ' + df['TIMEFROM'])

# Display the first few rows of the dataframe
df.head()

,PROFILE,DATEFROM,TIMEFROM,VALUE,UNIT,DATETIME
0,740105,2023.03.01,00:00:00,0.880,KWH,2023-03-01 00:00:00
1,740105,2023.03.01,00:15:00,0.968,KWH,2023-03-01 00:15:00
2,740105,2023.03.01,00:30:00,0.906,KWH,2023-03-01 00:30:00
3,740105,2023.03.01,00:45:00,0.916,KWH,2023-03-01 00:45:00
4,740105,2023.03.01,01:00:00,0.977,KWH,2023-03-01 01:00:00


## 2. Cleaning the Data

In [4]:
# Step 1: Handle Missing Values
# In this case, we'll drop rows with any missing values. 
# Alternatively, you can fill missing values with df.fillna() if appropriate.
df_cleaned = df.dropna()

# Step 2: Remove Duplicates
# Assuming a combination of PROFILE, DATEFROM, and TIMEFROM uniquely identifies each record,
# we remove duplicates based on these columns.
df_cleaned = df_cleaned.drop_duplicates(subset=['PROFILE', 'DATEFROM', 'TIMEFROM'])

# Step 3: Correct Data Types
# The 'VALUE' column should be numeric. Let's ensure this.
df_cleaned['VALUE'] = pd.to_numeric(df_cleaned['VALUE'], errors='coerce')

# Handle possible conversion issues by dropping rows with NaNs in 'VALUE' after conversion.
df_cleaned = df_cleaned.dropna(subset=['VALUE'])

# Now, let's correct the datetime conversion as before but now with the cleaned data
df_cleaned['DATETIME'] = pd.to_datetime(df_cleaned['DATEFROM'] + ' ' + df_cleaned['TIMEFROM'])

# Display the first few rows of the cleaned dataframe to verify
df_cleaned.head()

## check missing datapoints



,PROFILE,DATEFROM,TIMEFROM,VALUE,UNIT,DATETIME
0,740105,2023.03.01,00:00:00,0.880,KWH,2023-03-01 00:00:00
1,740105,2023.03.01,00:15:00,0.968,KWH,2023-03-01 00:15:00
2,740105,2023.03.01,00:30:00,0.906,KWH,2023-03-01 00:30:00
3,740105,2023.03.01,00:45:00,0.916,KWH,2023-03-01 00:45:00
4,740105,2023.03.01,01:00:00,0.977,KWH,2023-03-01 01:00:00


## 3. Analyze the Data

### Total Energy Consumption

In [5]:
total_energy_consumption = df['VALUE'].sum()
print(f"Total Energy Consumption: {total_energy_consumption} KWH")

Total Energy Consumption: 864343.2250000003 KWH


### 3.1 Data Preparation

In [6]:
# Group by day and sum the energy values
daily_data = df_cleaned.set_index('DATETIME').resample('D')['VALUE'].sum().reset_index()


### 3.2 Feature Engineering

In [7]:
# Add day of the week and weekend as features
daily_data['day_of_week'] = daily_data['DATETIME'].dt.dayofweek
daily_data['is_weekend'] = daily_data['day_of_week'].isin([5, 6]).astype(int)


### 3.3 Splitting the dataset

Split the data into training and test sets to ensure the model can be validated independently.

In [8]:
from sklearn.model_selection import train_test_split

# Define features and target
X = daily_data[['day_of_week', 'is_weekend']]
y = daily_data['VALUE']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### 3.4 Model Selection

In [9]:
from sklearn.ensemble import RandomForestRegressor

# Initialize and train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

### 3.5 Model Evaluation

Evaluate your model using metrics such as Mean Absolute Error (MAE) or Root Mean Square Error (RMSE).

In [10]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate RMSE and MAE
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")


RMSE: 1137.3127748788997
MAE: 948.3846727099744
